In [401]:
# compare ensemble to each baseline classifier
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble.gradient_boosting import GradientBoostingClassifier
from sklearn.ensemble.forest import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import StackingClassifier
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
import pandas as pd
from catboost import CatBoostClassifier, Pool, cv
import math
import numpy as np
from scipy.stats import uniform as sp_randFloat
from scipy.stats import randint as sp_randInt
def deg_to_rad(dr):
    return (dr*math.pi)/180


In [402]:
df = pd.read_csv('train.csv')
df['label'] = df['label'].map({'correct': 1, 'incorrect': 0})
df = df.dropna()
df = df.drop(['pickup_time','drop_time'], axis = 1)

df['pick_lat'] = deg_to_rad(df['pick_lat'])
df['pick_lon'] = deg_to_rad(df['pick_lon'])
df['drop_lat'] = deg_to_rad(df['drop_lat'])
df['drop_lon'] = deg_to_rad(df['drop_lon'])
dlon = df['drop_lon'] - df['pick_lon']
dlat = df['drop_lat'] - df['pick_lat']
df['dist'] = np.sin(dlat/2)**2 + np.cos(df['pick_lat'])*np.cos(df['drop_lat'])*np.sin(dlon/2)**2
df['dist'] = 2*np.arctan2(df['dist']**0.5,(1-df['dist'])**0.5)
R = 6373.0
df['dist'] = R*df['dist']

In [403]:
df = df.drop(['tripid','pick_lat','pick_lon','drop_lat','drop_lon'], axis = 1) 

In [404]:
y = df['label']
X = df.drop(['label'],axis=1)

In [405]:
from sklearn.model_selection import RandomizedSearchCV
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

model = CatBoostClassifier()
parameters = {'depth'         : sp_randInt(4, 10),
                  'learning_rate' : sp_randFloat(),
                  'iterations'    : sp_randInt(10, 100)
                 }

randm = RandomizedSearchCV(estimator=model, param_distributions = parameters,
                               cv = 2, n_iter = 10, n_jobs=-1)
randm.fit(X_train, y_train)

0:	learn: 0.4242416	total: 9.94ms	remaining: 457ms
1:	learn: 0.3221770	total: 13.4ms	remaining: 302ms
2:	learn: 0.2699511	total: 15.3ms	remaining: 224ms
3:	learn: 0.2349281	total: 17.5ms	remaining: 189ms
4:	learn: 0.2197491	total: 20.1ms	remaining: 169ms
5:	learn: 0.2105565	total: 22ms	remaining: 150ms
6:	learn: 0.1971796	total: 24ms	remaining: 137ms
7:	learn: 0.1932597	total: 26ms	remaining: 127ms
8:	learn: 0.1883176	total: 29.4ms	remaining: 124ms
9:	learn: 0.1843020	total: 31.8ms	remaining: 118ms
10:	learn: 0.1756270	total: 35.5ms	remaining: 116ms
11:	learn: 0.1719642	total: 37.7ms	remaining: 110ms
12:	learn: 0.1693353	total: 40.3ms	remaining: 106ms
13:	learn: 0.1669991	total: 42.5ms	remaining: 100ms
14:	learn: 0.1636777	total: 46.8ms	remaining: 99.9ms
15:	learn: 0.1623029	total: 50.8ms	remaining: 98.4ms
16:	learn: 0.1607798	total: 52.6ms	remaining: 92.9ms
17:	learn: 0.1597780	total: 54.5ms	remaining: 87.8ms
18:	learn: 0.1587684	total: 56.8ms	remaining: 83.7ms
19:	learn: 0.1579830	to

RandomizedSearchCV(cv=2,
                   estimator=<catboost.core.CatBoostClassifier object at 0x7fcacbe2eb38>,
                   n_jobs=-1,
                   param_distributions={'depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fcacbe2eb70>,
                                        'iterations': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fcacbdc4470>,
                                        'learning_rate': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fcacbd6e748>})

In [406]:
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
preds = randm.predict(X_test)
print("F1 score = {}".format(f1_score(y_test, preds)))

F1 score = 0.9775309265337034


In [407]:
df_test = pd.read_csv('test.csv')
df_test['pick_lat'] = deg_to_rad(df_test['pick_lat'])
df_test['pick_lon'] = deg_to_rad(df_test['pick_lon'])
df_test['drop_lat'] = deg_to_rad(df_test['drop_lat'])
df_test['drop_lon'] = deg_to_rad(df_test['drop_lon'])
dlon = df_test['drop_lon'] - df_test['pick_lon']
dlat = df_test['drop_lat'] - df_test['pick_lat']
df_test['dist'] = np.sin(dlat/2)**2 + np.cos(df_test['pick_lat'])*np.cos(df_test['drop_lat'])*np.sin(dlon/2)**2
df_test['dist'] = 2*np.arctan2(df_test['dist']**0.5,(1-df_test['dist'])**0.5)
R = 6373.0
df_test['dist'] = R*df_test['dist']
df_test =df_test.drop(['pickup_time','drop_time','pick_lat','pick_lon','drop_lat','drop_lon'], axis = 1)
df_new = df_test[:]
df_test = df_test.drop(['tripid'], axis = 1)
X_2 = df_test
y_ran_pred=randm.predict(X_2)
df_new['prediction'] = y_ran_pred

In [408]:
df_new = df_new.drop(['meter_waiting_till_pickup','additional_fare','duration','meter_waiting','meter_waiting_fare','fare','dist'], axis = 1) 

In [409]:
df_new.to_csv("prediction_results_cat.csv",index=False)

In [410]:
df_new

,tripid,prediction
0,213284604,1
1,213286352,1
2,213293973,1
3,213294622,1
4,213298687,1
...,...,...
8571,222856243,1
8572,222857785,1
8573,222858416,1
8574,222858691,1


In [411]:
df_new['prediction'].value_counts()

1    8235
0     341
Name: prediction, dtype: int64